## Imports

In [36]:
# DO NOT CHANGE THESE LINES.
using Suppressor
@suppress begin
    using DataFrames
    using LazyJSON
    using GLM 
    using MLJ 
    using MLJBase
    using CSV
    using Serialization
    using MLJScientificTypes
    using CategoricalArrays
    using JLD2
end


## Paths

In [37]:
# DO NOT CHANGE THESE LINES 
ROOT_DIR = dirname(pwd())
MODEL_INPUTS_OUTPUTS = joinpath(ROOT_DIR, "model_inputs_outputs")
INPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = joinpath(INPUT_DIR, "schema")
DATA_DIR = joinpath(INPUT_DIR, "data")
OUTPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "outputs")
TRAIN_DIR = joinpath(DATA_DIR, "training")
TEST_DIR = joinpath(DATA_DIR, "testing")
MODEL_PATH = joinpath(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = joinpath(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = joinpath(MODEL_ARTIFACTS_PATH, "ohe.jld2")
PREDICTOR_DIR_PATH = joinpath(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = joinpath(PREDICTOR_DIR_PATH, "predictor.jld2")
IMPUTATION_FILE = joinpath(MODEL_ARTIFACTS_PATH, "imputation.ser")
TOP_CATEGORIES = joinpath(MODEL_ARTIFACTS_PATH, "top_categories.ser")
PREDICTIONS_DIR = joinpath(OUTPUT_DIR, "predictions")
PREDICTIONS_FILE = joinpath(PREDICTIONS_DIR, "predictions.csv")
TARGET_LEVELS = joinpath(MODEL_ARTIFACTS_PATH, "target_levels.ser")
TARGET_MAPPING = joinpath(MODEL_ARTIFACTS_PATH, "target_mapping.ser")


if !isdir(MODEL_ARTIFACTS_PATH)
    mkdir(MODEL_ARTIFACTS_PATH)
end
if !isdir(PREDICTOR_DIR_PATH)
    mkdir(PREDICTOR_DIR_PATH)
end

### Reading the schema

In [38]:
# Reading a schema from a JSON file and extracting features
file_name = first(filter(x -> endswith(x, "json"), readdir(INPUT_SCHEMA_DIR)))
schema_path = joinpath(INPUT_SCHEMA_DIR, file_name)
schema_string = read(schema_path, String)  # Read file content as a string
schema = LazyJSON.parse(schema_string)
features = schema["features"]

# Identifying numeric, categorical, and nullable features
numeric_features = String[]
categorical_features = String[]
nullable_features = String[]

for f in features
    if f["dataType"] == "CATEGORICAL"
        push!(categorical_features, f["name"])
    else
        push!(numeric_features, f["name"])
    end
    if f["nullable"]
        push!(nullable_features, f["name"])
    end
end

# Extracting ID and target features
id_feature = schema["id"]["name"]
target_feature = schema["target"]["name"]
target_classes = schema["target"]["classes"]

if length(target_classes) == 2
    negative_class = target_classes[1]
    positive_class = target_classes[2]
end



"1"

### Reading test data.

In [39]:
file_name = filter(x -> occursin(".csv", x), readdir(TEST_DIR))[1]
file_path = joinpath(TEST_DIR, file_name)
df = DataFrame(CSV.File(file_path))

,id,destination,passenger,weather,temperature,time,coupon
,Int64,String15,String15,String7,Int64,String7,String31
1,9849,Home,Partner,Rainy,55,6PM,Coffee House
2,8457,No Urgent Place,Friend(s),Sunny,80,6PM,Coffee House
3,10270,Work,Alone,Rainy,55,7AM,Bar
4,7852,Home,Alone,Snowy,30,6PM,Coffee House
5,2020,Home,Alone,Sunny,55,6PM,Bar
6,10076,No Urgent Place,Alone,Sunny,80,10AM,Coffee House
7,9611,Home,Partner,Sunny,30,10PM,Restaurant(<20)
8,5700,No Urgent Place,Friend(s),Sunny,80,6PM,Restaurant(<20)
9,11845,No Urgent Place,Friend(s),Sunny,80,6PM,Restaurant(<20)


## Data preprocessing
Note that when we work with testing data, we have to impute using the same values learned during training. This is to avoid data leakage.

In [40]:
imputation_values = open(deserialize, IMPUTATION_FILE)
for column in nullable_features
    df[!, Symbol(column)] .= coalesce.(df[!, Symbol(column)], get(imputation_values, string(column), missing))
end

# Saving the id column in a different variable
ids = df[!, Symbol(id_feature)]

# Dropping the id and target from the DataFrame
select!(df, Not([Symbol(id_feature)]))

,destination,passenger,weather,temperature,time,coupon
,String15,String15,String7,Int64,String7,String31
1,Home,Partner,Rainy,55,6PM,Coffee House
2,No Urgent Place,Friend(s),Sunny,80,6PM,Coffee House
3,Work,Alone,Rainy,55,7AM,Bar
4,Home,Alone,Snowy,30,6PM,Coffee House
5,Home,Alone,Sunny,55,6PM,Bar
6,No Urgent Place,Alone,Sunny,80,10AM,Coffee House
7,Home,Partner,Sunny,30,10PM,Restaurant(<20)
8,No Urgent Place,Friend(s),Sunny,80,6PM,Restaurant(<20)
9,No Urgent Place,Friend(s),Sunny,80,6PM,Restaurant(<20)


### Encoding
We encode the data using the same encoder that we saved during training.

In [41]:
loaded_top_categories = open(deserialize, TOP_CATEGORIES)

# Function to one-hot encode only the top 10 categories
function one_hot_top_categories!(df, top_categories)
    for (feature, top_cats) in top_categories
        if length(top_cats) == 2  # Handle the binary case
            # Assuming the first category in top_cats is treated as 'true'
            new_col_name = "$(feature)_binary"
            df[!, new_col_name] = df[!, feature] .== top_cats[1]
        else  # Handle the general case
            for cat in top_cats
                new_col_name = "$(feature)_$(cat)"
                df[!, new_col_name] = df[!, feature] .== cat
            end
        end
        select!(df, Not(Symbol(feature)))  # Drop the original feature column
    end
end



one_hot_top_categories!(df, loaded_top_categories)

### Making predictions
Using the model saved during training.

In [42]:
# Load the saved model
loaded_mapping = open(deserialize, TARGET_MAPPING)
model = open(deserialize, PREDICTOR_FILE_PATH)

# Make predictions
log_odds_predictions = GLM.predict(model, convert(Matrix{Float64}, Matrix(df)))
probabilities = 1 ./ (1 .+ exp.(-log_odds_predictions))
threshold = 0.5
integer_predictions = ifelse.(probabilities .> threshold, 1, 0)
label_predictions = [loaded_mapping[pred] for pred in integer_predictions]
target_levels = open(deserialize, TARGET_LEVELS)
label_predictions = target_levels[integer_predictions .+ 1]


2537-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

### Creating predictions DataFrame.

In [43]:
result_df = DataFrame()

result_df[!, Symbol(negative_class)] = 1 .- probabilities
result_df[!, Symbol(positive_class)] = probabilities
result_df[!, id_feature] = ids
CSV.write(PREDICTIONS_FILE, result_df)
result_df

,0,1,id
,Float64,Float64,Int64
1,0.5,0.5,9849
2,0.5,0.5,8457
3,0.5,0.5,10270
4,0.5,0.5,7852
5,0.5,0.5,2020
6,0.5,0.5,10076
7,0.5,0.5,9611
8,0.5,0.5,5700
9,0.5,0.5,11845
